# Defining high-quality repeat catalogs

## Authors
- Ben Weisburd, Broad Institute
- Egor Dolzhenko, PacBio

## Introduction

Tandem repeats are regions of the genome that consist of consecutive copies of some motif sequence. For example, `CAGCAGCAG` is a tandem repeats with motif `CAG`. Many types of genomic studies require annotations of tandem repeats in the reference genome, called repeat catalogs. Repeat catalogs typically consist of the repeat reference coordinates and one or multiple motif sequences that the repeat is composed of.

The purpose of this tutorial is to explain how to define high-quality repeat catalogs. Our initial focus will be on the human genome, however the tutorial should also be applicable to other closely-related genomes. We'd also love to extend this work to plants and other species. Please consider creating a GitHub issue or reaching out by email if you are interested in this.

The high level strategy is described in [this AGBT poster](https://www.pacb.com/wp-content/uploads/AGBT-2024-Poster-Egor-Dolzhenko.pdf). To prepare for this analysis, do the following:
- Download the variation cluster analysis tool `vclust` from here: https://github.com/PacificBiosciences/vclust
- Obtain between 10 and 100 WGS HiFi BAM files with reads aligned to your target reference genome
- Install the latest version of bedtools: https://bedtools.readthedocs.io/en/latest/

## Step 1: Generate the core repeat catalog

The analysis steps are outlined in [a blog post here](https://bw2.github.io/2023-11-12-defining-genome-wide-TR-catalogs.html). See the technical details section.

## Step 2: Call variation cluster analysis for each repeat in the catalog

Call variation clusters which are informed by variation in all of your BAM files:

In [8]:
%%bash

./vclust --genome genome.fa --reads bams/*.bam --regions core_tr_catalog.bed > variation_clusters.txt

Generate a BED file with variation clusters:

In [15]:
%%bash

grep -v "NA" variation_clusters.txt \
    | awk '{OFS="\t"; print $5, $1}' \
    | awk -F "[\t:-]" '{OFS="\t"; print $1, $2, $3, $0}' \
    | cut -f 1-3,5 \
    | sort -k 1,1 -k 2,2n -k 3,3n \
    > raw_variation_clusters.bed

## Step 3: Merge variation clusters

Use bedtools to merge overlapping variation clusters:

In [36]:
recs = ! bedtools merge -c 4 -o collapse -i raw_variation_clusters.bed

Reformat the output into a valid TRGT repeat catalog (`variation_clusters.bed`):

In [37]:
def merge_names(names):
    if "," not in names:
        return names
    names = names.split(",")
    ids = []
    motifs = []
    for name in names:
        trid = name.split(";")[0].replace("ID=", "")
        tr_motifs = name.split(";")[1].replace("MOTIFS=", "").split(",")
        ids.append(trid)
        motifs.extend(tr_motifs)
    assert len(ids) == len(set(ids))
    ids = ",".join(ids)
    motifs = list(set(motifs))
    struc = "".join([f"({motif})n" for motif in motifs])
    motifs = ",".join(motifs)
    return f"ID={ids};MOTIFS={motifs};STRUC={struc}"


with open("variation_clusters.bed", "w") as file:
    for rec in recs:
        chrom, start, end, names = rec.split()
        name = merge_names(names)
        print(f"{chrom}\t{start}\t{end}\t{name}", file=file)

The variation clusters in `variation_clusters.bed` can now be genotyped with TRGT.